# Dataset Downloader

Notebook responsável por organizar os arquivos do dataset localmente

Carrega as informações referente aos arquivos de dataset presente em: https://physionet.org/content/chbmit/1.0.0/

Motivador: Devido ao tamanho dos arquivos e da conexão com a internet, a ideia é permitir baixar os arquivos separados e executar conforme haja tempo, permitindo retomar o download em casos de perca de conexão.

#### Pacotes Necessários

In [ ]:
!pip install requests
!pip install wfdb

Armazena cada um dos diretórios presente no dataset, utilizado posteriormente para fazer o donwload separado de cada arquivo

- record_list: Armazenar todos os arquivos dentro do dataset -> path/path-summary.txt + -> path/path_chxx.edf
- part_codes: Armazena os diretórios principais de cada exame

In [ ]:
import wfdb 

dbs = wfdb.get_dbs()
records_list = wfdb.io.get_record_list('chbmit', records='all')

part_codes = sorted(list(set([record.split('/')[0] for record in records_list])))
edf_files = sorted(list(set([record.split('/')[1] for record in records_list])))

get_summary: Faz o download de cada um dos summarios presentes nos exames e os adiciona ao path ../data/

In [ ]:
import os
from urllib.request import urlretrieve

def get_summary( part_code ):
  url = "https://physionet.org/physiobank/database/chbmit/" + part_code + '/' + part_code + '-summary.txt'
  directory = "../data/" + part_code + '/'
  filename = part_code + "-summary.txt"

  fullpath = os.path.join( directory, filename )

  if not os.path.exists( fullpath ):
    os.makedirs( directory )
    urlretrieve( url, fullpath )
  else:
    print(part_code + ": Já existe")
    
#Executa
for part_code in part_codes:
  get_summary( part_code )


get_edf_by_record: Baixar cada um dos arquivos edf's presente no dataset por exame. Arquivos já baixados, não são executados novamente

In [ ]:
import os
from urllib.request import urlretrieve

def get_edf_by_record(record, file):
  url = "https://physionet.org/physiobank/database/chbmit/" + record + '/' + file
  directory = "../data/" + record + '/'
  filename = file

  fullpath = os.path.join( directory, filename )

  if not os.path.exists( fullpath ):
    try:
      urlretrieve( url, fullpath )
      print(fullpath + ": Baixado!")
    except:
      print( filename + ": Não foi possivel, verificar")
    
  else:
    print(fullpath + ": Já existe!")
    
#Executa
for record in part_codes:
    files_by_record = list( filter( lambda file: record + '_' in file, edf_files ) )
    for file in files_by_record:
        get_edf_by_record( record, file )